In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.optimizers import SGD, adam
import scipy.io as sio
import loss.my_loss as my_loss

Using TensorFlow backend.


In [2]:
training_set = {}
training_set['received'] = sio.loadmat('.\\data\\denoise_set\\sps_4\\train_received')['received_set']
training_set['clean'] = sio.loadmat('.\\data\\denoise_set\\sps_4\\train_clean')['clean_set']
training_set['snr'] = sio.loadmat('.\\data\\denoise_set\\sps_4\\train_snr')['snr_set']

In [3]:
x = np.zeros([training_set['received'].shape[0],2,training_set['received'].shape[1]])
x[:,0,:]=np.real(training_set['received'])
x[:,1,:]=np.imag(training_set['received'])
y = np.zeros([training_set['received'].shape[0],2,training_set['received'].shape[1]])
y[:,0,:]=np.real(training_set['clean'])
y[:,1,:]=np.imag(training_set['clean'])
x = x.astype(np.float32)
y = y.astype(np.float32)

In [4]:
out_shape = list(x.shape)
inp_shape = np.append(out_shape, 1)
x = x.reshape(inp_shape)
y = y.reshape(inp_shape)
print(x.shape)

(20000, 2, 1024, 1)


In [5]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, TensorBoard, EarlyStopping
lr_decay = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, verbose=1, min_lr=1e-6, mode='min')
early_stopping = EarlyStopping(monitor='loss', patience=10, verbose=1, mode='min')
callback_list = [lr_decay, early_stopping] 

In [6]:
import model.ddn as my_model
model = my_model.u_net_side_fuse()
# model.summary()

D:\python-wing-personal\script\MR_4\model\ddn.py:114: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (1, 4), padding="same", kernel_initializer="he_normal", strides=(1, 1), activation=None, name="input_layer")`
  name='input_layer', padding='same', activation=None)(inp)
D:\python-wing-personal\script\MR_4\model\ddn.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, padding="same", kernel_initializer="he_normal", kernel_size=(1, 1), strides=(1, 1), activation=None, name="s1_block_conv_1")`
  name=name+'_conv_1', padding='same', activation=None)(inp)
D:\python-wing-personal\script\MR_4\model\ddn.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, padding="same", kernel_initializer="he_normal", kernel_size=(1, 4), strides=(1, 1), activation=None, name="s1_block_conv_2")`
  name=name+'_conv_2', padding='same', activation=None)(out)
D:\python-wing-personal\script\MR_4\model\ddn.py:37: UserWarning: Update your `Conv

In [7]:
from loss.my_loss import sqrt_evm_loss,evm_loss
model.compile(loss={'o1': sqrt_evm_loss,
                    'o2': sqrt_evm_loss,
                    'o3': sqrt_evm_loss,
                    'o_fuse': sqrt_evm_loss},
                      optimizer='adam',
                      metrics={'o_fuse':evm_loss})

In [8]:
x_train = x[:18000]
y_train = y[:18000]
x_val = x[18000:]
y_val = y[18000:]

In [9]:
history1 = model.fit(x=x_train, y=[y_train,y_train,y_train,y_train],
                    epochs=100,
                    batch_size=32,
                    verbose=1,
                    validation_data=(x_val, [y_val,y_val,y_val,y_val]),
                    callbacks=callback_list)
model.save_weights('.\\weights\\uNet_sqrtevmloss.h5')

Train on 18000 samples, validate on 2000 samples
Epoch 1/100
18000/18000 [==============================] - 90s - loss: 38.1510 - o1_loss: 8.0540 - o2_loss: 10.8500 - o3_loss: 11.4778 - o_fuse_loss: 7.7692 - o_fuse_evm_loss: 291.8332 - val_loss: 38.5998 - val_o1_loss: 8.0661 - val_o2_loss: 13.4037 - val_o3_loss: 12.6551 - val_o_fuse_loss: 4.4750 - val_o_fuse_evm_loss: 49.5991
Epoch 2/100
18000/18000 [==============================] - 84s - loss: 23.0813 - o1_loss: 4.1026 - o2_loss: 7.1769 - o3_loss: 8.0036 - o_fuse_loss: 3.7981 - o_fuse_evm_loss: 31.0508 - val_loss: 20.5841 - val_o1_loss: 3.4356 - val_o2_loss: 6.2289 - val_o3_loss: 7.5316 - val_o_fuse_loss: 3.3881 - val_o_fuse_evm_loss: 21.9299
Epoch 3/100
18000/18000 [==============================] - 83s - loss: 18.5411 - o1_loss: 3.3757 - o2_loss: 5.4753 - o3_loss: 6.3405 - o_fuse_loss: 3.3497 - o_fuse_evm_loss: 23.1572 - val_loss: 16.8972 - val_o1_loss: 2.9244 - val_o2_loss: 5.0888 - val_o3_loss: 5.9585 - val_o_fuse_loss: 2.9255 - 

18000/18000 [==============================] - 84s - loss: 16.4830 - o1_loss: 2.9679 - o2_loss: 4.8033 - o3_loss: 5.7439 - o_fuse_loss: 2.9679 - o_fuse_evm_loss: 20.5903 - val_loss: 15.4207 - val_o1_loss: 2.6244 - val_o2_loss: 4.6025 - val_o3_loss: 5.5693 - val_o_fuse_loss: 2.6246 - val_o_fuse_evm_loss: 12.0632
Epoch 48/100
17984/18000 [============================>.] - ETA: 0s - loss: 16.4458 - o1_loss: 2.9563 - o2_loss: 4.7952 - o3_loss: 5.7381 - o_fuse_loss: 2.9563 - o_fuse_evm_loss: 20.3508
Epoch 00047: reducing learning rate to 1e-06.
18000/18000 [==============================] - 84s - loss: 16.4533 - o1_loss: 2.9577 - o2_loss: 4.7974 - o3_loss: 5.7406 - o_fuse_loss: 2.9576 - o_fuse_evm_loss: 20.3734 - val_loss: 15.3672 - val_o1_loss: 2.6032 - val_o2_loss: 4.5949 - val_o3_loss: 5.5660 - val_o_fuse_loss: 2.6032 - val_o_fuse_evm_loss: 11.7660
Epoch 49/100
18000/18000 [==============================] - 84s - loss: 16.4370 - o1_loss: 2.9506 - o2_loss: 4.7957 - o3_loss: 5.7400 - o_fus

In [10]:
model_2 = my_model.u_net_side_fuse()
model_2.compile(loss={'o1': evm_loss,
                    'o2': evm_loss,
                    'o3': evm_loss,
                    'o_fuse': evm_loss},
                      optimizer='adam',
                      metrics={'o_fuse':evm_loss})
history2 = model_2.fit(x=x_train, y=[y_train,y_train,y_train,y_train],
                    epochs=100,
                    batch_size=32,
                    verbose=1,
                    validation_data=(x_val, [y_val,y_val,y_val,y_val]),
                    callbacks=callback_list)
model_2.save_weights('.\\weights\\uNet_evmloss.h5')

D:\python-wing-personal\script\MR_4\model\ddn.py:114: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (1, 4), padding="same", kernel_initializer="he_normal", strides=(1, 1), activation=None, name="input_layer")`
  name='input_layer', padding='same', activation=None)(inp)
D:\python-wing-personal\script\MR_4\model\ddn.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, padding="same", kernel_initializer="he_normal", kernel_size=(1, 1), strides=(1, 1), activation=None, name="s1_block_conv_1")`
  name=name+'_conv_1', padding='same', activation=None)(inp)
D:\python-wing-personal\script\MR_4\model\ddn.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, padding="same", kernel_initializer="he_normal", kernel_size=(1, 4), strides=(1, 1), activation=None, name="s1_block_conv_2")`
  name=name+'_conv_2', padding='same', activation=None)(out)
D:\python-wing-personal\script\MR_4\model\ddn.py:37: UserWarning: Update your `Conv

Train on 18000 samples, validate on 2000 samples
Epoch 1/100
18000/18000 [==============================] - 88s - loss: 1593.7845 - o1_loss: 332.6278 - o2_loss: 467.4800 - o3_loss: 533.1331 - o_fuse_loss: 260.5435 - o_fuse_evm_loss: 260.5435 - val_loss: 736.6846 - val_o1_loss: 85.2945 - val_o2_loss: 235.3414 - val_o3_loss: 302.5946 - val_o_fuse_loss: 113.4541 - val_o_fuse_evm_loss: 113.4541
Epoch 2/100
18000/18000 [==============================] - 84s - loss: 399.5411 - o1_loss: 55.4089 - o2_loss: 112.3936 - o3_loss: 167.2855 - o_fuse_loss: 64.4531 - o_fuse_evm_loss: 64.4531 - val_loss: 221.7397 - val_o1_loss: 23.9819 - val_o2_loss: 64.2225 - val_o3_loss: 101.1143 - val_o_fuse_loss: 32.4210 - val_o_fuse_evm_loss: 32.4210
Epoch 3/100
18000/18000 [==============================] - 83s - loss: 211.6007 - o1_loss: 26.5415 - o2_loss: 62.0120 - o3_loss: 91.1202 - o_fuse_loss: 31.9269 - o_fuse_evm_loss: 31.9269 - val_loss: 162.0910 - val_o1_loss: 16.3953 - val_o2_loss: 51.3380 - val_o3_loss: